<a href="https://colab.research.google.com/github/fonteleslrivka/Emergence-of-Causal-Order-from-a-Pre-Geometric-Substrate/blob/main/Utility_Functions_(stable_softmax%2C_safe_eigs%2C_etc_).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===========================================================
# 2. Utilities: stable softmax, safe eigen solvers
# ===========================================================

def stable_softmax(evals, clip_min=-100, clip_max=100):
    ev = np.asarray(evals, dtype=np.float64)
    ev = np.clip(ev, clip_min, clip_max)
    m = np.max(ev)
    z = np.exp(ev - m)
    Z = np.sum(z)
    if not np.isfinite(Z) or Z == 0:
        lam = np.ones_like(ev) / ev.size
    else:
        lam = z / Z
    lam = np.clip(lam, 1e-300, 1.0)
    lam /= lam.sum()
    return lam


def safe_eigsh_dense_or_sparse(mat, k=40, which='LA'):
    n = mat.shape[0]

    if n <= 800:
        try:
            return eigh(mat, eigvals_only=True)
        except Exception:
            pass

    try:
        if not isinstance(mat, csr_matrix):
            M = csr_matrix(mat)
        else:
            M = mat

        k_try = min(max(6, min(k, n-2)), n-2)
        vals = eigsh(M, k=k_try, which=which, return_eigenvectors=False)
        return np.asarray(vals)

    except:
        return eigh(mat, eigvals_only=True)
